# Quantum Galton Box - Interactive Exploration

This notebook provides an interactive environment for exploring the Quantum Galton Box implementation based on the paper "Universal Statistical Simulator" by Carney & Varcoe (2022).

In [ ]:
# Setup and imports
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from quantum_galton_box import QuantumGaltonBox
from simulation_runner import GaltonBoxSimulator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from qiskit.visualization import circuit_drawer

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Basic Circuit Visualization

In [ ]:
# Create and visualize a simple 2-layer circuit
qgb = QuantumGaltonBox(num_layers=2)
circuit = qgb.generate_optimized_circuit()

# Display circuit
print("2-Layer Quantum Galton Box Circuit:")
circuit.draw('mpl', style='iqx')

## 2. Interactive Circuit Explorer

In [ ]:
def explore_circuit(num_layers):
    """Interactive function to explore circuits of different sizes."""
    qgb = QuantumGaltonBox(num_layers)
    circuit = qgb.generate_optimized_circuit()
    
    # Get metrics
    metrics = qgb.get_circuit_metrics()
    
    # Display information
    print(f"\n{num_layers}-Layer Quantum Galton Box")
    print("="*40)
    print(f"Qubits: {metrics['num_qubits']}")
    print(f"Circuit Depth: {metrics['circuit_depth']}")
    print(f"Total Gates: {metrics['total_gates']}")
    print(f"Gate Breakdown: {metrics['gate_counts']}")
    print(f"Efficiency: {metrics['efficiency']:.2%}")
    
    # Draw circuit if small enough
    if num_layers <= 3:
        return circuit.draw('mpl', style='iqx')
    else:
        print("\n(Circuit too large to display)")

# Create interactive widget
layer_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=5,
    step=1,
    description='Layers:',
    continuous_update=False
)

widgets.interact(explore_circuit, num_layers=layer_slider)

## 3. Simulation and Distribution Analysis

In [ ]:
def simulate_and_analyze(num_layers, shots):
    """Run simulation and analyze the distribution."""
    qgb = QuantumGaltonBox(num_layers)
    circuit = qgb.generate_optimized_circuit()
    
    # Run simulation
    results = qgb.simulate(shots=shots)
    
    # Verify Gaussian
    verification = qgb.verify_gaussian(plot=True)
    
    # Print statistics
    print(f"\nStatistics for {num_layers}-layer circuit with {shots} shots:")
    print(f"Mean: {results['mean']:.4f} (Theory: {verification['theoretical_mean']:.4f})")
    print(f"Std: {results['std']:.4f} (Theory: {verification['theoretical_std']:.4f})")
    print(f"KS p-value: {verification['ks_pvalue']:.4f}")
    print(f"Gaussian: {'Yes' if verification['is_gaussian'] else 'No'}")
    
    return results, verification

# Interactive widgets
layer_widget = widgets.IntSlider(value=4, min=1, max=10, description='Layers:')
shots_widget = widgets.IntSlider(value=5000, min=1000, max=20000, step=1000, description='Shots:')

widgets.interact(simulate_and_analyze, num_layers=layer_widget, shots=shots_widget)

## 4. Scaling Analysis

In [ ]:
# Run scaling analysis for multiple layer counts
print("Running scaling analysis...")
simulator = GaltonBoxSimulator(layer_range=list(range(1, 8)), shots=5000)
df = simulator.run_all_simulations(verbose=True)

# Display results table
display(df)

In [ ]:
# Generate scaling plots
simulator.plot_scaling_analysis(df)

## 5. Custom Analysis Functions

In [ ]:
def compare_methods(num_layers=3):
    """Compare general vs optimized circuit generation."""
    qgb = QuantumGaltonBox(num_layers)
    
    # Generate both versions
    print(f"Comparing methods for {num_layers} layers:")
    print("="*40)
    
    # General method
    general_circuit = qgb.generate_galton_circuit()
    qgb.circuit = general_circuit
    general_metrics = qgb.get_circuit_metrics()
    
    # Optimized method
    optimized_circuit = qgb.generate_optimized_circuit()
    qgb.circuit = optimized_circuit
    optimized_metrics = qgb.get_circuit_metrics()
    
    # Create comparison table
    comparison_data = {
        'Metric': ['Qubits', 'Circuit Depth', 'Total Gates', 'Efficiency'],
        'General': [
            general_metrics['num_qubits'],
            general_metrics['circuit_depth'],
            general_metrics['total_gates'],
            f"{general_metrics['efficiency']:.2%}"
        ],
        'Optimized': [
            optimized_metrics['num_qubits'],
            optimized_metrics['circuit_depth'],
            optimized_metrics['total_gates'],
            f"{optimized_metrics['efficiency']:.2%}"
        ]
    }
    
    comparison_df = pd.DataFrame(comparison_data)
    display(comparison_df)
    
    # Calculate improvement
    gate_reduction = (general_metrics['total_gates'] - optimized_metrics['total_gates']) / general_metrics['total_gates']
    depth_reduction = (general_metrics['circuit_depth'] - optimized_metrics['circuit_depth']) / general_metrics['circuit_depth']
    
    print(f"\nOptimization Results:")
    print(f"Gate reduction: {gate_reduction:.1%}")
    print(f"Depth reduction: {depth_reduction:.1%}")

compare_methods(3)

In [ ]:
def batch_statistical_test(max_layers=6, runs_per_layer=5):
    """Run multiple simulations to test statistical consistency."""
    results_data = []
    
    for n_layers in range(1, max_layers + 1):
        print(f"Testing {n_layers} layers...")
        qgb = QuantumGaltonBox(n_layers)
        circuit = qgb.generate_optimized_circuit()
        
        for run in range(runs_per_layer):
            results = qgb.simulate(shots=5000)
            verification = qgb.verify_gaussian(plot=False)
            
            results_data.append({
                'layers': n_layers,
                'run': run + 1,
                'mean': results['mean'],
                'std': results['std'],
                'ks_pvalue': verification['ks_pvalue'],
                'is_gaussian': verification['is_gaussian']
            })
    
    # Create DataFrame and analyze
    batch_df = pd.DataFrame(results_data)
    
    # Group statistics
    grouped = batch_df.groupby('layers').agg({
        'mean': ['mean', 'std'],
        'std': ['mean', 'std'],
        'ks_pvalue': ['mean', 'min', 'max'],
        'is_gaussian': 'mean'
    }).round(4)
    
    print("\nStatistical Consistency Analysis:")
    print("="*60)
    display(grouped)
    
    # Plot consistency
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Mean consistency
    for n_layers in range(1, max_layers + 1):
        layer_data = batch_df[batch_df['layers'] == n_layers]
        axes[0].scatter([n_layers] * len(layer_data), layer_data['mean'], alpha=0.5)
    
    axes[0].set_xlabel('Number of Layers')
    axes[0].set_ylabel('Measured Mean')
    axes[0].set_title('Mean Consistency Across Runs')
    axes[0].grid(True, alpha=0.3)
    
    # P-value distribution
    for n_layers in range(1, max_layers + 1):
        layer_data = batch_df[batch_df['layers'] == n_layers]
        axes[1].scatter([n_layers] * len(layer_data), layer_data['ks_pvalue'], alpha=0.5)
    
    axes[1].axhline(y=0.05, color='r', linestyle='--', label='α=0.05')
    axes[1].set_xlabel('Number of Layers')
    axes[1].set_ylabel('KS Test p-value')
    axes[1].set_title('Gaussian Test Consistency')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    return batch_df

batch_results = batch_statistical_test(max_layers=5, runs_per_layer=3)

## 6. Export and Save Results

In [ ]:
# Export simulation results
if 'df' in locals():
    df.to_csv('../results/notebook_results.csv', index=False)
    print("Results exported to ../results/notebook_results.csv")
    
    # Generate summary report
    print("\nSummary Report:")
    print("="*60)
    print(f"Total simulations: {len(df)}")
    print(f"Layer range: {df['layers'].min()} to {df['layers'].max()}")
    print(f"All Gaussian tests passed: {df['is_gaussian'].all()}")
    print(f"Average KS p-value: {df['ks_pvalue'].mean():.4f}")
    print(f"Mean absolute error: {(df['mean'] - df['theoretical_mean']).abs().mean():.4f}")
    print(f"Std absolute error: {(df['std'] - df['theoretical_std']).abs().mean():.4f}")